In [8]:
import pandas as pd
import numpy as np
from datetime import datetime
import xlrd
import pickle
import calendar
import os
from dotenv import dotenv_values, find_dotenv

# this looks for your configuration file and then reads it as a dictionary
config = dotenv_values(find_dotenv())

# set path using the dictionary key for which one you want
path_cleandata = os.path.abspath(config["CLEANDATA"]) + '\\'
path_rawdata = os.path.abspath(config["RAWDATA"]) + '\\'

In [3]:
# Import forward rates
fwdwide = pd.read_pickle(path_cleandata+'BR_Fwd_dM'+'.pkl')

# Import spot rates
spotwide = pd.read_pickle(path_cleandata+'BR_Spot_dM'+'.pkl')

# Import CPI
cpiwide = pd.read_pickle(path_cleandata+'DataStream_CPI_M'+'.pkl')

In [4]:
# convert spot data from wide to long
spotlong = pd.melt(spotwide.reset_index(), id_vars='index', var_name='imfcode', value_name='spot').rename(columns={'index': 'Date'})

# convert forward data from wide to long
fwdlong = pd.melt(fwdwide.reset_index(), id_vars='index', var_name='imfcode', value_name='fwd1').rename(columns={'index': 'Date'})

# convert cpi data from wide to long
cpilong = pd.melt(cpiwide.reset_index(), id_vars='EndOfMonth', var_name='imfcode', value_name='cpi').rename(columns={"EndOfMonth": "Date"})

In [5]:
# merge spot and forward data
fxdat = pd.merge(left=spotlong, right=fwdlong, how='inner', on=['Date', 'imfcode'])

In [6]:
# merge CPI and FX
# will return an empty dataset since fake CPI data has no common dates with FX data
alldata = pd.merge(left=cpilong, right=fxdat, how='outer', on=['Date', 'imfcode'])

In [9]:
# add iso3 column
alldata['iso3'] = np.nan

# change imf codes to iso3: euro area
alldata.loc[alldata['imfcode'] == '163', 'iso3'] = 'EUR'

# change imf codes to iso3: other countries
# make an imf to iso code dictionary
IMF_to_ISO = pd.read_csv(path_rawdata+'Country_iso_codes.csv')
# imf codes in data are stored as strings
IMF_to_ISO['IMF'] = IMF_to_ISO['IMF'].astype(str)
# create dictionary from xlsx file with imf to iso codes
IMF_to_ISO_dict = dict(zip(IMF_to_ISO['IMF'], IMF_to_ISO['iso3']))
# fill in the iso3 column for the rest of the countries
alldata['iso3'] = alldata['iso3'].fillna(alldata['imfcode'].map(IMF_to_ISO_dict))

In [10]:
# create us-only cpi dataframe
cpius = alldata.loc[alldata['imfcode'] == '111'][['Date', 'cpi']]
cpius = cpius.rename(columns={'cpi': 'cpius'})


In [11]:
# create merge cpius with alldata
alldata = pd.merge(left=alldata, right=cpius, how='inner', on=['Date'])

In [11]:
# reset index
# alldata.set_index('Date', inplace=True)

# fix the way the dates are displaying
# alldata.index = pd.to_datetime(alldata.index, format='%Y-%m-%d')

# change index name to date
# alldata.index.names = ['Date']

In [12]:
# save
alldata[['iso3', 'Date', 'spot', 'fwd1', 'cpi', 'cpius']].to_pickle(path_cleandata+'BR_Spot_Fwd_Real'+'.pkl')